## 밑바닥 딥러닝
## 2019.08.02 
### 기울기, 경사하강법에 대해서는 Winter Workshop 2019, Andrew Ng, Introduction to Machine Learning 자료 참조
### 4.4.2. 신경망에서의 기울기 
### 4.5. 학습 알고리즘 구현하기
### 4.5.1. 2층 신경망 구현하기 
### 4.5.2. 미니배치 학습 구현하기
### 4.5.3. 시험 데이터로 평가하기
### 4.6. 정리 

[복습]

왜 손실함수인가? 

- 신경망 학습에서는 '최적의 매개변수(가중치와 편향)'을 탐색할 때 손실함수의 값을 가능한 작게 하는 매개변수 값을 찾는다. 

매개 변수의 미분을 계산하고 그 미분값을 단서로 매개변수 값을 서서히 갱신하는 과정을 반복한다. 

신경망을 학습할 때 정확도를 지표로 삼아서는 안 된다. 정확도를 지표로 삼을 경우 매개변수의 미분의 대부분의 장소에서 0이 된다. 

대신 손실함수를 지표로 삼으면 매개변수 값이 조금 변하면 그에 반응하여 연속적으로 변화한다. 

계단함수의 미분은 대부분의 장소 (0이외의 곳)에서 0이다. 



In [0]:
# 그림 4-4 계단함수와 시그모이드 함수: 계단 함수는 대부분의 장소에서 기울기가 0이지만, 시그모이드 함수의 기울기(접선)는 0이 아니다.
import numpy as np
import matplotlib.pylab as plt

def step_function(x):
    return np.array(x > 0, dtype=np.int)

x = np.arange(-6.0, 6.0, 0.1)
y1 = step_function(x)
y2 = np.array([1 for _ in range(x.size)])
y3 = np.array([0 for _ in range(x.size)])
plt.plot(x, y1)
plt.plot(x, y2, color='green')
plt.plot(x, y3, color='green')
plt.scatter([4,-4],[1,0],color='red')

plt.ylim(-0.1, 1.1) # y축의 범위 지정
plt.show()

In [0]:
#시그모이드 함수의 기울기 (접선) = 0이 아님을 확인 가능
#출처: https://nbviewer.jupyter.org/github/SDRLurker/deep-learning/blob/master/4%EC%9E%A5.ipynb

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_diff(x):
    return sigmoid(x) * (1 - sigmoid(x))

def sigmoid_tangent(x): # 접선 ax+b에서 a,b 값을 리턴
    return sigmoid_diff(x), sigmoid(x) - sigmoid_diff(x) * x

x = np.arange(-6.0, 6.0, 0.1)
y1 = sigmoid(x)
a2, b2 = sigmoid_tangent(4)
y2 = a2 * x + b2
a3, b3 = sigmoid_tangent(-4)
y3 = a3 * x + b3
plt.plot(x, y1)
plt.plot(x, y2, color='green')
plt.plot(x, y3, color='green')
plt.scatter([4,-4],[a2*4+b2,a3*-4+b3],color='red')

plt.ylim(-0.1, 1.1) # y축의 범위 지정
plt.show()

계단 함수는 한 순간만 변화를 일으키지만, 시그모이드 함수는 출력이 연속적으로 변하고 곡선의 기울기(미분)도 연속적으로 변함. 기울기가 0이 되지 않는 덕분에 신경망이 올바르게 학습된다. 

## 수치 미분
## 편미분 
## 기울기 
## 경사법(경사 하강법)

-- 복습을 위한 참고 자료 : 

- Andrew Ng (Introduction to ML, Chapter 02, Coursera)
- MIT Deep Learning Lecture (#1) https://www.youtube.com/watch?v=5v1JnYv_yWs
- Gradient Descent and Cost Function : [medium](https://towardsdatascience.com/machine-learning-fundamentals-via-linear-regression-41a5d11f5220?fbclid=IwAR3inkeUx3yadKlkau4wUBJlgR1ceO2Pqlz6uVuIKtvDz_xp4-VVEWG8uWc)

### 4.4.2. 신경망에서의 기울기 

신경망 학습에서도 기울기를 구해야한다. 여기서 말하는 기울기는 가중치 매개변수에 대한 손실 함수의 기울기이다. 예를 들어 형상이 2*3,  가중치가 W, 손실함수가  L인 신경망을 생각해보자. 이 경우 경사는 &theta;L/&theta;W 로 나타낼 수 있다. 수식으로는 다음과 같다. 

\begin{equation*}
W =  \begin{vmatrix}
w_{11} w_{21} w_{31}\\
w_{12} w_{22} w_{32}\
\end{vmatrix}
\end{equation*}


\begin{equation*}
\frac{\partial{L}}{\partial{W}} =  \begin{vmatrix}
\frac{\partial{L}}{\partial{W_{11}}} \frac{\partial{L}}{\partial{W_{21}}} \frac{\partial{L}}{\partial{W_{31}}}\\
\frac{\partial{L}}{\partial{W_{12}}} \frac{\partial{L}}{\partial{W_{22}}} \frac{\partial{L}}{\partial{W_{32}}}\
\end{vmatrix}
\end{equation*}

&theta;L/&theta;W 의 각 원소는 각각의 원소에 관한 편미분이다. 

예를 들어 1행 1번째 원소인 &theta;L/&theta;W<sub>11</sub>은 w<sub>11</sub>을 조금 변경했을 때 손실 함수 L이 얼마나 변화하느냐를 나타낸다. 여기서 중요한 점은 θL/θW의 형상이 W와 같다는 것이다. 실제로 [식 4.8]에서 W와 &theta;L/&theta;W의 형상은 모두 2*3이다. 

간단한 신경망을 예로 들어 실제로 기울기를 구하는 코드를 구현해보자. 

In [0]:
import sys, os 
sys.path.append(os.pardir)
import numpy as np
from common.functions import softmax, cross_entropy_error
from common.gradient import numerical_gradient

class simpleNet:
  def __init__(self):
    self.W = np.random.randn(2,3) #정규분포로 초기화
    
  def predict(self, x):
    return np.dot(x, self.W)
  
  
  def loss(self, x, t):
    z = self.predict(x)
    y = softmax(z)
    loss = cross_entropy_error(y, t)
    
    return loss
  

In [0]:
!git clone https://github.com/WegraLee/deep-learning-from-scratch.git

여기에서는 common/functions.py에 정의한 softmax와 cross_entropy_error 메서드를 사용한다. 

In [0]:
!ls

In [0]:
import os
os.chdir("./deep-learning-from-scratch")

simpleNet 클래스는 형상이 2*3인 가중치 매개변수 하나를 인스턴스 변수로 갖는다. 메서드는 2개인데, 하나는 예측을 수행하는 predict(x)이고, 다른 하나는 손실 함수의 값을 구하는 loss(x, t)이다. 여기에서 인수 x는 입력 데이터, t는 정답 레이블이다. 그럼 simpleNet을 사용해 몇 가지 시험을 해보자. 

In [0]:
net = simpleNet()

print(net.W) #가중치 매개변수

In [0]:
x = np.array([0.6, 0.9])
p = net.predict(x)

In [0]:
print(p)

In [0]:
np.argmax(p)

In [0]:
t = np.array([0, 0, 1]) #정답 레이블

In [0]:
net.loss(x, t)

이어서 기울기를 구해볼까요? 지금까지처럼 numerical_gradient(f, x)를 써서 구하면 된다. (여기에서 정의한 f(W) 함수의 인수 W는 더미<sup>dummy</sup>로 만든 것이다) numerical_gradient(f, x) 내부에서 f(x)를 실행하는데, 그와의 일관성을 위해 f(W)를 정의한 것이다. 

In [0]:
def f(W):
  return net.loss(x, t)

In [0]:
dW = numerical_gradient(f, net.W)

In [0]:
print(dW)

numerical_gradient(f, x)의 인수 f는 함수, x는 함수f의 인수이다. 그래서 여기에서는 net.W를 인수로 받아 손실 함수를 계산하는 새로운 함수f를 정의했다. 그리고 이 새로 정의한 함수를 numerical_gradient(f, x)에 넘긴다. 

dW는 numerical_gradient(f, net.W)의 결과로, 그 형상은 2*3의 2차원 배열이다. dW의 내용을 보면, 예를 들어  θL/θW의 θL/θW<sub>11</sub>은 대략 0.2이다. 

이는 w<sub>11</sub>을 _h_ 만큼 늘리면 손실 함수의 값은 0.2h만큼 증가한다. 마찬가지로 θL/θW<sub>23</sub>은 대략 -0.5이니 θL/θW<sub>23</sub>을 h만큼 늘리면 손실 함수의 값은 0.5h만큼 감소하는 것이다. 그래서 손실 함수를 줄인다는 관점에서는 θL/θW<sub>23</sub>은 양의 방향으로 갱신하고 w<sub>11</sub>은 음의 방향으로 갱신해야함을 알 수 있다. 또, 한 번에 갱신되는 양에는 θL/θW<sub>23</sub>이 w<sub>11</sub>보다 크게 기여한다는 사실도 알 수 있다. 



참고로 이 구현에서는 새로운 함수를 정의하는 데 "def f(x).."문법을 썼는데 파이썬에서는 간단한 함수라면 람다(lambda)기법을 쓰면 더 편하다. 

In [0]:
f = lambda w: net.loss(x, t)
dW = numerical_gradient(f, net.W)

In [0]:
print(f)

In [0]:
dW

신경망의 기울기를 구한 다음에는 경사법에 따라 가중치 매개변수를 갱신하기만 하면 된다. 다음 절에서는 2층 신경망을 대상으로 학습 과정 전체를 구현한다. 

### 4.5. 학습 알고리즘 구현하기

신경망 학습에 관한 기본적인 지식에 대해 배웠다. 
"손실 함수", "미니 배치", "기울기", "경사 하강법" 등의 중요한 키워드에 대해 배웠다. 

신경망 학습의 순서를 다시 짚어보자. 

- 전체
  - 신경망에는 적응 가능한 가중치와 편향이 있고, 이 가중치와 편향을 훈련 데이터에 적응하도록 조정하는 과정을 '학습'이라 한다. 신경망 학습은 다음과 같이 4단계로 수행한다. 
  
  - 1단계 : 미니배치
    훈련 데이터 중 일부를 무작위로 가져온다. 이렇게 선별한 데이터를 미니배치라 하며, 그 미니배치의 손실 함수 값을 줄이는 것이 목표다 
    
  - 2단계: 기울기 산출
    미니배치의 손실 함수 값을 줄이기 위해 각 가중치 매개 변수의 기울기를 구한다. 기울기는 손실 함수의 값을 가장 작게 하는 방향을 제시한다. 
    
   - 3단계: 매개 변수 갱신
      가중치 매개변수를 기울기 방향으로 아주 조금 갱신한다. 
      
   - 4단계: 반복
   
     1~3 단계를 반복한다. 


이 것이 신경망 학습이 이뤄지는 순서다. 이는 경사 하강법으로 매개 변수를 갱신하는 방법이며, 이때 데이터를 미니배치로 무작위로 선정하기 때문에 **확률적 경사 하강법**(stochastic gradient descent)이라고 부른다. "확률적으로 무작위로 골라낸 데이터"에 대해 수행하는 경사 하강법이라는 의미다. 
대부분의 딥러닝 프레임워크는 확률적 경사 하강법의 영어 머리글자를 딴  SGD라는 함수로 이 기능을 구현한다. 

실제로 손글씨 숫자를 학습하는 신경망을 구현해보자. 
여기에서는 2층 신경망(은닉층이 1개인 네트워크)을 대상으로 MNIST 데이터셋을 사용하여 학습을 수행한다. 


   

### 4.5.1. 2층 신경망 클래스 구현하기

처음에는 2층 신경망을 하나의 클래스로 구현하는 것부터 시작한다. 이 클래스의 이름은 TwoLayerNet이다. 

소스코드 : ch04/two_layer_net.py

In [0]:
# https://github.com/WegraLee/deep-learning-from-scratch/blob/master/ch04/two_layer_net.py 소스 참고
import numpy as np

def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x : 입력 데이터, t : 정답 레이블
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
    
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

TwoLayerNet 클래스가 사용하는 변수

params : 신경망의 매개변수를 보관하는 딕셔너리

params['W1'] : 1번째 층의 가중치, params['b1'] : 1번째 층의 편향

params['W2'] : 2번째 층의 가중치, params['b2'] : 2번째 층의 편향

grads : 기울기를 보관하는 딕셔너리

grads['W1'] : 1번째 층의 가중치, grads['b1'] : 1번째 층의 편향

grads['W2'] : 2번째 층의 가중치, grads['b2'] : 2번째 층의 편향

TwoLayerNet 클래스의 메서드

__init__(self, input_size, hidden_size, output_size) : 초기화를 수행

input_size: 입력층의 뉴런수, hidden_size: 은닉층의 뉴런수, output_size: 출력층의 뉴런수

predict(self, x) : 예측(추론)을 수행. 인수 x는 이미지 데이터

loss(self, x, t) : 손실 함수의 값을 구함. x는 이미지 데이터. t는 정답 레이블(나머지 메소드 인수도 동일)

accuracy(self, x, t) : 정확도를 구함

numerical_gradient(self, x, t) : 가중치 매개변수의 기울기를 구함

gradient(self, x, t) : 가중치 매개변수의 기울기를 구함. numerical_gradient의 성능 개선판! (구현은 다음장에서...)



### 1층의 매개변수 예시 


In [0]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)
print(net.params['W1'].shape) # (784, 100)
print(net.params['b1'].shape) # (100,)
print(net.params['W2'].shape) # (100, 10)
print(net.params['b2'].shape) # (10,)

params변수에는 신경망에 필요한 매개변수가 모두 저장된다. 

그리고 params 변수에 저장된 가중치 매개변수가 예측 처리 (순방향 처리)에서 사용된다. 참고로 예측 처리는 다음과 같이 실행할 수 있다. 



In [0]:
x = np.random.rand(100, 784) #더미 입력 데이터 (100장 분량)

y = net.predict(x)

In [0]:
def sigmoid(x):
  return 1/ (1 + np.exp(-x))

grad변수에는 params변수에 대응하는 각 매개변수의 기울기가 저장된다. 
예를 들어 다음과 같이 numerical_gradient()메서드를 사용해 기울기를 계산하면 grads변수에 기울기 정보가 저장된다. 

In [0]:
x = np.random.rand(100, 784) #더미 입력 데이터 (100장 분량)
t = np.random.rand(100, 10) #더미 정답 레이블 (100장 분량)

grads = net.numerical_gradient(x, t) #기울기 계산

grads['W1'].shape #(784, 100)
grads['b1'].shape #(100, )
grads['W2'].shape #(100, 10)
grads['b2'].shape #(10, )

초기화 메서드 : __init__ (self, input_size, hidden_size, output_size) 메서드 
(순서대로 입력층의 뉴런 수, 은닉층의 뉴런 수, 출력층의 뉴런 수)

예를 들어 손글씨 숫자 인식에서는 크기가 28*28인 입력 이미지가 총 784개이고, 출력은 10개가 된다. 따라서 input_size = 784, output_size = 10로 지정하고 은닉층의 개수인 hidden_size는 적당한 값을 설정한다. 


이 초기화 메서드에서는 가중치 매개변수도 초기화/ 가중치 매개변수의 초깃값을 무엇으로 설정하냐가 신경망 학습의 성공을 좌우하기도 한다. 가중치 매개변수 초기화에 대한 내용은 나중에 재방문. 



우선, 가중치를 정규분포를 따르는 난수, 편향을 0으로  가정하고 초기화함.

loss(self, x, t) = 손실 함수 계산

numerical_gradient(self, x, t) 메서드는 각 매개변수의 기울기를 계산함, 수치 미분 방식으로 각 매개변수의 손실 함수에 대한 기울기를 계산

gradient(self, x, t) 메서드: 다음장에서 재방문/ **오차역전차법**을 사용하여 기울기를 효율적이고 빠르게 계산

### 4.5.2. 미니배치 학습 구현하기

미니배치에 대해 경사법으로 매개변수를 갱신한다. 그럼 TwoLayerNet클래스와 MNIST 데이터셋을 사용하여 학습을 수행한다 (소스 코드: ch04/train_neuralnet.py)

In [0]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist


# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
#이후 추가
#이후 추가

# 1에폭당 반복 수
#이후 추가

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    #이후 추가

여기서는 미니배치 크기를 100으로 설정함 (즉, 매번 60,000개의 훈련 데이터에서 임의로 100개의 데이터 (이미지, 정답)를 추려낸다)

그런 다음 그 100개의 미니배치를 대상으로 확률적 경사 하강법을 수행해 매개변수를 갱신한다. 

경사법에 의한 갱신 횟수 (반복 횟수)를 10,000번으로 설정하고, 갱신할 때마다 훈련 데이터에 대한 손실 함수를 계산하고, 그 값을 배열에 추가한다. 

이 손실 함수의 값이 변화하는 추이를 그래프로 나타내면 [그림 4-11]처럼 된다. 

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
# 그림 4-11 손실 함수의 추이: 위쪽은 10,000회 반복까지의 추이, 아래쪽은 1,000회 반복까지의 추이
f, (ax1, ax2) = plt.subplots(2, 1)
x = np.array(range(iters_num))
ax1.plot(x, train_loss_list, label='loss')
ax1.set_xlabel("iteration")
ax1.set_ylabel("loss")
ax1.set_ylim(0, 3.0)
ax2.plot(x[:1000], train_loss_list[:1000], label='loss')
ax2.set_xlabel("iteration")
ax2.set_ylabel("loss")
ax2.set_ylim(0, 3.0)

[그림 4-11]를 보면 학습 횟수가 늘어가면서 손실 함수의 값이 줄어든다.  =신경망의 가중치 매개변수가 서서히 데이터에 적응하고 있음을 의미

이는신경망의 학습이 잘 되고 있다는 뜻이다.  

 = 데이터를 반복해서 학습함으로써 최적 가중치 매개변수로 서서히 다가서고 있다!

### 4.5.3. 시험 데이터로 평가하기

[그림 4-11]의 결과에서 학습을 반복함으로써 손실 함수의 값이 서서히 내려가는 것을 확인해보았다. 

이때의 손실 함수의 **값**이란, 정확히는 '훈련 데이터의 미니배치에 대한 손실 함수'의 값이다. 훈련 데이터의 손실 함수 값이 작아지는 것은 신경망이 잘 학습하고 있다는 방증이지만 이 결과만으로는 다른 데이터셋에도 비슷한 실력을 발휘할지는 확실하지 않다!

신경망 학습에서는 훈련 데이터 외의 데이터를 올바르게 인식하는지를 확인해야한다. (즉, **오버피팅**을 일으키지 않는지를 확인해야한다)

**오버피팅**이 되었다는 것은 훈련데이터에 포함된 이미지만 제대로 구분하고, 그렇지 않은 이미지는 식별할 수 없다는 뜻이다. 



시험 데이터로 신경망이 훈련 데이터에 포함되지 않았던 *새* 데이터에도 적용 가능한지 확인해보자. 여기에서는 1 에폭 별로 훈련 데이터와 시험 데이터에 대한 정확도를 기록한다. 

**에폭(epoch)** : 하나의 단위, 1 에폭은 학습에서 훈련 데이터를 모두 소진했을 때의 횟수에 해당한다. 예컨데 훈련 데이터 10,000개를 100개의 미니배치로 학습할 경우, 확률적 경사 하강법을 100회 반복하면 모든 훈련 데이터를 '소진'한 게 된다. 이 경우 100회가 1 에폭이 된다. 

In [0]:
# 데이터 읽기
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼파라미터
iters_num = 10000  # 반복 횟수를 적절히 설정한다.
train_size = x_train.shape[0]
batch_size = 100   # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    # 기울기 계산
    #grad = network.numerical_gradient(x_batch, t_batch)
    grad = network.gradient(x_batch, t_batch)
    
    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]
    
    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

이 예시에서는 1 에폭마다 모든 훈련 데이터와 시험 데이터에 대한 정확도를 계산하고, 그 결과를 기록한다. 정확도를 1 에폭마다 계산하는 이유는 for문 안에서 매번 계산하기에는 시간이 오래 걸리고, 또 그렇게까지 자주 기록할 필요도 없기 때문이다. 더 큰 관점에서 그 추이를 알 수 있으면 충분하다. 



아래 [그림 4-12]에서 훈련 데이터에 대한 정확도를 실선으로, 시험 데이터에 대한 정확도를 점선으로 그렸다. 

보다시피 에폭이 진행될수록 (학습이 진행될수록) 훈련 데이터와 시험 데이터를 사용하고 평가한 정확도가 모두 좋아지고 있다. 

또, 두 정확도에는 차이가 없음을 알 수 있다 (거의 겹쳐있음) = 이번 학습에서는 오버피팅이 일어나지 않았다. 

만약 오버피팅이 일어났다면 어느 순간부터 시험 데이터에 대한 정확도가 떨어지기 시작한다. (= 이럴 때, 학습을 중단하는데 이걸 조기 종료 (early stopping)라고 하며 이후 "6.4. 바른 학습을 위해" 장에서 살펴볼 "가중치 감소", "드롭 아웃"과 함께 배울 대표적인 오버피팅 예방책이다 )

In [0]:
# 그림 4-12 훈련 데이터와 시험 데이터에 대한 정확도 추이
markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

### 4.6. 정리 

이번 장에서 배운 내용

- 기계 학습에서 사용하는 데이터셋은 훈련 데이터와 시험 데이터로 나눠 사용한다
- 훈련 데이터로 학습한 모델의 범용 능력을 시험 데이터로 평가한다
- 신경망 학습은 손실 함수를 지표로, 손실 함수의 값이 작아지는 방향으로 가중치 매개변수를 갱신한다
- 가중치 매개변수를 갱신할 때는 가중치 매개변수의 기울기를 이용하고, 기울어진 방향으로 가중치의 값을 갱신하는 작업을 반복한다. 
- 아주 작은 값을 주었을 때의 차분으로 미분하는 것을 수치 미분이라고 한다. 
- 수치 미분을 이용해 가중치 매개변수의 기울기를 구할 수 있다. 
- 수치 미분을 이용한 계산에는 시간이 걸리지만, 그 구현은 간단하다. 한편, 다음 장에서 구현하는 (다소 복잡한) 오차역전파법은 기울기를 고속으로 구할 수 있다. 

